### Q2 - Tabu search

In [ ]:
import pandas as pd
import random as rd
from itertools import combinations
import math

''' 基本變數 static '''
num_job=20 # number of jobs

j=list(range(1,num_job+1)) 
p=[10,10,13,4,9,4,8,15,7,1,9,3,15,9,11,6,5,14,18,3]
d=[50,38,49,12,20,105,73,45,6,64,15,6,92,43,78,21,15,50,150,99]
w=[10,5,1,5,10,1,5,10,5,1,5,10,10,5,1,10,5,5,1,5]

data = pd.DataFrame({
    # 'Job': j,
    'weight': w,
    "processing_time": p,
    "due_date": d,
}, index=j)
data

,weight,processing_time,due_date
1,10,10,50
2,5,10,38
3,1,13,49
4,5,4,12
5,10,9,20
6,1,4,105
7,5,8,73
8,10,15,45
9,5,7,6
10,1,1,64


In [ ]:
# pd.read_excel('Instance_20.xlsx', names=['Job', 'weight', "processing_time", "due_date"],
#                                  index_col=0).to_dict('index')

In [ ]:


class TS():
    def __init__(self, data, seed, tabu_tenure):
        # self.Path = Path
        self.seed = seed
        self.tabu_tenure = tabu_tenure
        self.instance_dict = data.to_dict('index')
        # self.instance_dict = self.input_data()
        self.Initial_solution = self.get_InitialSolution()
        self.tabu_str, self.Best_solution, self.Best_objvalue = self.TSearch()


    def input_data(self):
        '''Takes the path of the excel file of the SMTWTP instances.
        Returns a dict of jobs number as Key and weight, processing time (hours) and due date (hours) as values.
        '''
        return pd.read_excel(self.Path, names=['Job', 'weight', "processing_time", "due_date"],
                                 index_col=0).to_dict('index')

    def get_tabuestructure(self):
        '''Takes a dict (input data)
        Returns a dict of tabu attributes(pair of jobs that are swapped) as keys and [tabu_time, MoveValue]
        '''
        dict = {}
        for swap in combinations(self.instance_dict.keys(), 2):
            dict[swap] = {'tabu_time': 0, 'MoveValue': 0}
        return dict

    def get_InitialSolution(self, show=False):
        n_jobs = len(self.instance_dict) # Number of jobs
        # Producing a random schedule of jobs
        initial_solution = list(range(1, n_jobs+1))
        rd.seed(self.seed)
        rd.shuffle(initial_solution)
        if show == True:
            print("initial Random Solution: {}".format(initial_solution))
        return initial_solution

    def Objfun(self, solution, show = False):
        '''Takes a set of scheduled jobs, dict (input data)
        Return the objective function value of the solution
        '''
        dict = self.instance_dict
        t = 0   #starting time
        objfun_value = 0
        for job in solution:
            C_i = t + dict[job]["processing_time"]  # Completion time
            d_i = dict[job]["due_date"]   # due date of the job
            T_i = max(0, C_i - d_i)    #tardiness for the job
            W_i = dict[job]["weight"]  # job's weight

            objfun_value +=  W_i * T_i
            t = C_i
        if show == True:
            print("\n","="*10, "The Objective function value for {} solution schedule is: {}".format(solution ,objfun_value),"="*10)
        return objfun_value

    def SwapMove(self, solution, i ,j):
        '''Takes a list (solution)
        returns a new neighbor solution with i, j swapped
       '''
        solution = solution.copy()
        # job index in the solution:
        i_index = solution.index(i)
        j_index = solution.index(j)
        #Swap
        solution[i_index], solution[j_index] = solution[j_index], solution[i_index]
        return solution

    def TSearch(self):
        '''The implementation Tabu search algorithm with short-term memory and pair_swap as Tabu attribute.
        '''
        # Parameters:
        tenure =self.tabu_tenure
        tabu_structure = self.get_tabuestructure()  # Initialize the data structures
        best_solution = self.Initial_solution
        best_objvalue = self.Objfun(best_solution)
        current_solution = self.Initial_solution
        current_objvalue = self.Objfun(current_solution)

        print("="*30, "Short-term memory TS with Tabu Tenure: {}\nInitial Solution: {}, Initial Objvalue: {}".format(
            tenure, current_solution, current_objvalue), "="*30, sep='\n\n')
        iter = 1
        Terminate = 0
        while Terminate < 100:
            print('\n\n# iter {}  Current_Objvalue: {}, Best_Objvalue: {}'.format(iter, current_objvalue,
                                                                                    best_objvalue))
            # Searching the whole neighborhood of the current solution:
            for move in tabu_structure:
                candidate_solution = self.SwapMove(current_solution, move[0], move[1])
                candidate_objvalue = self.Objfun(candidate_solution)
                tabu_structure[move]['MoveValue'] = candidate_objvalue

            # Admissible move
            while True:
                # select the move with the lowest ObjValue in the neighborhood (minimization)
                best_move = min(tabu_structure, key =lambda x: tabu_structure[x]['MoveValue'])
                MoveValue = tabu_structure[best_move]["MoveValue"]
                tabu_time = tabu_structure[best_move]["tabu_time"]
                # Not Tabu
                if tabu_time < iter:
                    # make the move
                    current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = self.Objfun(current_solution)
                    # Best Improving move
                    if MoveValue < best_objvalue:
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("   best_move: {}, Objvalue: {} => Best Improving => Admissible".format(best_move,
                                                                                                      current_objvalue))
                        Terminate = 0
                    else:
                        print("   #Termination: {}# best_move: {}, Objvalue: {} => Least non-improving => "
                              "Admissible".format(Terminate,best_move,
                                                                                                           current_objvalue))
                        Terminate += 1
                    # update tabu_time for the move
                    tabu_structure[best_move]['tabu_time'] = iter + tenure
                    iter += 1
                    break
                # If tabu
                else:
                    # Aspiration
                    if MoveValue < best_objvalue:
                        # make the move
                        current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                        current_objvalue = self.Objfun(current_solution)
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("   best_move: {}, Objvalue: {} => Aspiration => Admissible".format(best_move,
                                                                                                      current_objvalue))
                        Terminate = 0
                        iter += 1
                        break
                    else:
                        tabu_structure[best_move]["MoveValue"] = float('inf')
                        print("   best_move: {}, Objvalue: {} => Tabu => Inadmissible".format(best_move,
                                                                                              current_objvalue))
                        continue
        print('='*30 , "Performed iterations: {}".format(iter), "Best found Solution: {} , Objvalue: {}".format(best_solution,best_objvalue), sep="\n")
        return tabu_structure, best_solution, best_objvalue


#### Run Tabu Script
參數調整
- tabu_tenure = `3`
- tabu_tenure = `5`
- tabu_tenure = `10`

結論
- 不同的記憶長度，得到相同的最佳解 `Best found Solution: [12, 17, 4, 16, 5, 9, 8, 1, 11, 10, 7, 13, 20, 14, 2, 18, 6, 15, 3, 19] , Objvalue: 1928`，而且禁忌搜尋法只花了 20 回合就得到這組解。雖然基因演算法只要花 12 回合就可得到最佳解，如果迭迨需要花費大量的運算，那麼基因或許比較適合，但這個情境若要追求最佳解，那麼禁忌搜尋法更適合使用，畢竟 1928 還是比 21xx 低了 1/10 的延遲加權。

In [ ]:
tabuSearch = TS(data=data, seed = 2022, tabu_tenure=3)


Short-term memory TS with Tabu Tenure: 3
Initial Solution: [5, 16, 8, 20, 4, 17, 3, 11, 1, 6, 7, 14, 12, 13, 9, 2, 19, 15, 10, 18], Initial Objvalue: 4033



# iter 1  Current_Objvalue: 4033, Best_Objvalue: 4033
   best_move: (3, 12), Objvalue: 3296 => Best Improving => Admissible


# iter 2  Current_Objvalue: 3296, Best_Objvalue: 3296
   best_move: (8, 12), Objvalue: 2906 => Best Improving => Admissible


# iter 3  Current_Objvalue: 2906, Best_Objvalue: 2906
   best_move: (3, 18), Objvalue: 2629 => Best Improving => Admissible


# iter 4  Current_Objvalue: 2629, Best_Objvalue: 2629
   best_move: (6, 9), Objvalue: 2470 => Best Improving => Admissible


# iter 5  Current_Objvalue: 2470, Best_Objvalue: 2470
   best_move: (5, 12), Objvalue: 2350 => Best Improving => Admissible


# iter 6  Current_Objvalue: 2350, Best_Objvalue: 2350
   best_move: (9, 20), Objvalue: 2260 => Best Improving => Admissible


# iter 7  Current_Objvalue: 2260, Best_Objvalue: 2260
   best_move: (13, 18), Objvalue

In [ ]:
tabuSearch = TS(data=data, seed = 2022, tabu_tenure=5)


Short-term memory TS with Tabu Tenure: 5
Initial Solution: [5, 16, 8, 20, 4, 17, 3, 11, 1, 6, 7, 14, 12, 13, 9, 2, 19, 15, 10, 18], Initial Objvalue: 4033



# iter 1  Current_Objvalue: 4033, Best_Objvalue: 4033
   best_move: (3, 12), Objvalue: 3296 => Best Improving => Admissible


# iter 2  Current_Objvalue: 3296, Best_Objvalue: 3296
   best_move: (8, 12), Objvalue: 2906 => Best Improving => Admissible


# iter 3  Current_Objvalue: 2906, Best_Objvalue: 2906
   best_move: (3, 18), Objvalue: 2629 => Best Improving => Admissible


# iter 4  Current_Objvalue: 2629, Best_Objvalue: 2629
   best_move: (6, 9), Objvalue: 2470 => Best Improving => Admissible


# iter 5  Current_Objvalue: 2470, Best_Objvalue: 2470
   best_move: (5, 12), Objvalue: 2350 => Best Improving => Admissible


# iter 6  Current_Objvalue: 2350, Best_Objvalue: 2350
   best_move: (9, 20), Objvalue: 2260 => Best Improving => Admissible


# iter 7  Current_Objvalue: 2260, Best_Objvalue: 2260
   best_move: (13, 18), Objvalue

In [ ]:
tabuSearch = TS(data=data, seed = 2022, tabu_tenure=10)


Short-term memory TS with Tabu Tenure: 10
Initial Solution: [5, 16, 8, 20, 4, 17, 3, 11, 1, 6, 7, 14, 12, 13, 9, 2, 19, 15, 10, 18], Initial Objvalue: 4033



# iter 1  Current_Objvalue: 4033, Best_Objvalue: 4033
   best_move: (3, 12), Objvalue: 3296 => Best Improving => Admissible


# iter 2  Current_Objvalue: 3296, Best_Objvalue: 3296
   best_move: (8, 12), Objvalue: 2906 => Best Improving => Admissible


# iter 3  Current_Objvalue: 2906, Best_Objvalue: 2906
   best_move: (3, 18), Objvalue: 2629 => Best Improving => Admissible


# iter 4  Current_Objvalue: 2629, Best_Objvalue: 2629
   best_move: (6, 9), Objvalue: 2470 => Best Improving => Admissible


# iter 5  Current_Objvalue: 2470, Best_Objvalue: 2470
   best_move: (5, 12), Objvalue: 2350 => Best Improving => Admissible


# iter 6  Current_Objvalue: 2350, Best_Objvalue: 2350
   best_move: (9, 20), Objvalue: 2260 => Best Improving => Admissible


# iter 7  Current_Objvalue: 2260, Best_Objvalue: 2260
   best_move: (13, 18), Objvalu